In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import pathlib
from tensorflow import keras

In [2]:
def train_val_generators(TRAINING_DIR, TESTING_DIR):
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest'
                                      )
    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                       batch_size=15,
                                                       class_mode='categorical',
                                                       target_size=(150,150))
    
    validation_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
    validation_generator = validation_datagen.flow_from_directory(directory=TESTING_DIR,
                                                                 batch_size=5,
                                                                 class_mode='categorical',
                                                                 target_size=(150, 150))
    
    return train_generator, validation_generator

In [3]:
TRAINING_DIR = "../DATACATPEDIGREE/training/"
TESTING_DIR = "../DATACATPEDIGREE/testing/"
train_generator, validation_generator = train_val_generators(TRAINING_DIR, TESTING_DIR)

Found 108283 images belonging to 55 classes.
Found 12057 images belonging to 55 classes.


In [8]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(55, activation='softmax')
])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 17, 17, 128)     

In [5]:
# Set the training parameters
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [7]:
# Train the model
history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator,  validation_steps=3)

Epoch 1/25
20/20 [==============================] - 18s 873ms/step - loss: 2.5803 - accuracy: 0.4367 - val_loss: 2.3991 - val_accuracy: 0.5333
Epoch 2/25
20/20 [==============================] - 18s 873ms/step - loss: 2.6918 - accuracy: 0.4200 - val_loss: 2.3248 - val_accuracy: 0.4667
Epoch 3/25
20/20 [==============================] - 18s 874ms/step - loss: 2.4803 - accuracy: 0.4767 - val_loss: 1.7538 - val_accuracy: 0.6667
Epoch 4/25
20/20 [==============================] - 16s 812ms/step - loss: 2.6294 - accuracy: 0.4467 - val_loss: 2.8798 - val_accuracy: 0.4000
Epoch 5/25
20/20 [==============================] - 18s 884ms/step - loss: 2.6655 - accuracy: 0.4333 - val_loss: 2.5226 - val_accuracy: 0.4000
Epoch 6/25
20/20 [==============================] - 17s 849ms/step - loss: 2.5201 - accuracy: 0.4600 - val_loss: 2.0929 - val_accuracy: 0.6000
Epoch 7/25
20/20 [==============================] - 17s 846ms/step - loss: 2.5458 - accuracy: 0.4567 - val_loss: 3.4995 - val_accuracy: 0.2000

In [15]:
# Select mode of optimization
mode = "Speed" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [16]:
export_dir = 'model_1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: model_1\assets


In [17]:

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()

In [20]:
tflite_model_file = pathlib.Path('model_1.tflite')
tflite_model_file.write_bytes(tflite_model)

3515552

In [22]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()



25

In [27]:
import os

with open('labels.txt', 'w') as f:
    for label in os.listdir('../DATACATPEDIGREE/training/'):
        f.writelines(f'{label}\n')